# Initial Imports and constants

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import lightkurve as lk
from lightkurve import search_targetpixelfile
import pandas as pd
import os
import batman
from scipy.stats import norm
import matplotlib
import jax.numpy as jnp

In [19]:
### Define data directory
data_dir = "/Users/pentrican10/Projects/Data/k2-19-data"

TESS_offset = 2457000
petigura_offset = 2454833

### switch to mask out transits
mask_transits = True

### set range for search: [#hours] * [days per hour]
ttv_hour = 2* 0.0416667 # 1 hour to days

In [34]:
import os

working_dir = os.getcwd()
data_dir = os.path.join(working_dir, "data", "k2-19-data")
print(f"Data directory: {data_dir}")
print(f"Data directory: {data_dir}")

Data directory: /Users/pentrican10/Projects/k2-19-project


# BLS Fit

In [20]:
# Download the light curve data
lc = lk.search_lightcurve("K2-19",author = 'SPOC').download_all()

# Flatten the light curve
lc = lc.stitch().flatten(window_length=901).remove_outliers()
#lc.plot()
time = lc.time
flux=lc.flux
flux_err = lc.flux_err

In [21]:
### perform periodigram for planet b
### Create array of periods to search
period = np.linspace(1, 20, 10000)

### Create a BLSPeriodogram
bls = lc.to_periodogram(method='bls', period=period, frequency_factor=500)
#bls.plot()

period_b_bls = bls.period_at_max_power.value
tc_b_bls = bls.transit_time_at_max_power.value
dur_b_bls = bls.duration_at_max_power.value

print(f'Period from BLS: {period_b_bls}')
print(f'TC from BLS: {tc_b_bls}')

Period from BLS: 7.9204920492049204
TC from BLS: 2530.2807708159753


# TESS Transit Times

In [22]:
from scipy.optimize import minimize
from scipy.optimize import root_scalar
from scipy.optimize import least_squares
from scipy.optimize import curve_fit

In [23]:
### read table from Petigura et al. 2020
file = "ajab5220t1_mrt.txt"
def read_table(file_name):
    ### path to table - Petigura et al 2020
    file_path = os.path.join(data_dir, file_name)

    ### Define the column names 
    columns = ["Planet", "Transit", "Inst", "Tc", "e_Tc", "Source"]

    ### Read the text file, specifying space as the delimiter, skipping model_guess_omc rows
    df = pd.read_csv(file_path, delim_whitespace=True, skiprows=22, names=columns)

    ### Remove NaN values
    df = df.dropna()
    return df

df = read_table(file)

/var/folders/69/wlx91x8s1x34mphlfjy0h4pm0000gn/T/ipykernel_69768/3528285165.py:11: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True, skiprows=22, names=columns)


In [24]:
### params from exoplanet archive
per_b = 7.9222
rp_b = 0.0777
T14_b = 3.237 * 0.0416667  # convert to days
b_b = 0.17
q1_b = 0.4
q2_b = 0.3

### Get lightcurve data from TESS

In [25]:
### function to convert times from TESS to Petigura offset
TESS_offset = 2457000
petigura_offset = 2454833
def convert_time_t2p(times):
    ### TESS offset 
    BTJD = times + TESS_offset
    new_time = BTJD - petigura_offset
    return new_time

In [26]:
### Download the light curve data
lc = lk.search_lightcurve("K2-19",author = 'SPOC').download_all()
lc = lc.stitch()
if mask_transits == True:
    ### mask transit times before flattening
    #transit_times = [4697.28834658, 4713.12428017, 4721.03972171, 4728.96021376, 4736.88070581, 4744.79614735, 5433.87895563, 5449.71993973]
    transit_times = [4697.28834658, 4713.12933068, 4721.03972171, 4728.96021376, 4736.88070581, 4744.80119786, 5433.87895563, 5449.71993973]
    masked_lc = lc

    times = convert_time_t2p(masked_lc.time.value)

    ### Initialize a mask with all False values (i.e., include all data points initially)
    mask = np.zeros_like(times, dtype=bool)

    ### Iterate through each transit time and update the mask
    for transit_time in transit_times:
        mask |= (times > (transit_time - T14_b/2)) & (times < (transit_time + T14_b/2))

    ### Flatten the masked light curve
    masked_lc = masked_lc.flatten(window_length=901, mask=mask).remove_outliers()
    lc = masked_lc
else:
    ### flatten unmasked lightcurve 
    lc = lc.flatten(window_length=901).remove_outliers()

### Fit for transit times

In [27]:
### initialize guess times (using BLS ephem)
transit_num = [0,2,3,4,5,6,93,95]

tc_guess=[]
for num in transit_num:
    t = tc_b_bls + (num * period_b_bls)
    tc_guess.append(t)

### data from lightcurve 
time_tess = np.array(lc.time.value)
flux=np.array(lc.flux)
flux_err = np.array(lc.flux_err)

time = convert_time_t2p(time_tess)
tc_guess = convert_time_t2p(np.array(tc_guess))
tc_guess = np.array(tc_guess)
print(f'TC guess(TESS): {tc_guess}')

TC guess(TESS): [4697.28077082 4713.12175491 4721.04224696 4728.96273901 4736.88323106
 4744.80372311 5433.88653139 5449.72751549]


In [28]:
### get tc ranges for fit
tc = []
for i in range(len(tc_guess)):
    start = tc_guess[i] - ttv_hour
    end = tc_guess[i] + ttv_hour
    t = np.linspace(start,end, 1000)
    tc.append(t)

In [29]:
### initialize arrays
tc_chi = np.zeros(len(tc))
tc_chi_parabola = np.zeros(len(tc))
ttv = np.zeros(len(tc))
ttv_p = np.zeros(len(tc))
errors = []
errors_p = []

In [30]:
def omc(obs_time, t_num, p, tc):
    calc_time = tc + (t_num* p)
    omc = obs_time - calc_time
    return omc#*24 #days to hours

### Find the intersection points
def intersection_func(t): #masked
    return np.interp(t, tc1, chi_sq) - err_threshold

In [31]:
### plot X^2 vs tc for each guess
for j in range(len(tc)):
    tc1 = tc[j]
    chi_sq = np.zeros(len(tc1))
    chi_sq_lc = np.zeros(len(tc1))
    for i in range(len(tc1)):
        t0_b = 	tc1[i]
        theta_initial = [t0_b, per_b, rp_b, b_b, T14_b, q1_b, q2_b]
        
        ### initialize params
        params = batman.TransitParams()
        params.t0, params.per, params.rp,params.b, params.T14, q1, q2 = theta_initial
        params.u = [2*np.sqrt(q1)*q2, np.sqrt(q1)*(1-2*q2)]  # Limb darkening coefficients
        params.limb_dark = 'quadratic'
        
        ### mask data - extract relevant photometry
        start = tc_guess[j] - ttv_hour
        end = tc_guess[j] + ttv_hour
        mask = (time > (start)) & (time < (end))
        
        transit_model = batman.TransitModel(params, time[mask])
            
        # Generate model light curve
        model_flux = transit_model.light_curve(params)
        
        # Calculate chi-squared value
        sigma2 = flux_err[mask] 
        chi_squared = np.sum(((flux[mask] - model_flux) / sigma2)**2)
        chi_sq[i] = (chi_squared)

    ### masked
    min_chi_time = tc1[np.argmin(chi_sq)]
    min_chi = chi_sq.min()

    tc_chi[j] = min_chi_time
    idx = transit_num[j]
    ttv[j] = min_chi_time - tc_guess[j]

    chi_mask = (chi_sq <= min_chi + 3)
    fit_mask = (chi_sq <= min_chi + 1)

    ### fit parabola to the chisq
    p_chi_sq = np.polyfit(tc1[fit_mask], chi_sq[fit_mask], 2)  

    ### Extract the coefficients   y = ax^2 + bx + c
    a_chi_sq, b_chi_sq, c_chi_sq = p_chi_sq
    
    ### Find the minimum of the parabola xmin = -b/2a from taking derivative=0
    tc_best_fit = -b_chi_sq / (2 * a_chi_sq)
    
    ### Calculate the minimum chi-squared value
    chi_sq_min = a_chi_sq * tc_best_fit**2 + b_chi_sq * tc_best_fit + c_chi_sq
    tc_chi_parabola[j] = tc_best_fit

    ### Calculate the parabola best fit 
    p_1 = a_chi_sq*tc1**2 + b_chi_sq*tc1 + c_chi_sq

    ### calculate ttv from parabola fit 
    ttv_p[j] = tc_best_fit - tc_guess[j]

    ### delta chisq = 1 gives errors
    err_threshold = min_chi + 1 # using chisq discrete minimum
    err_threshold_p = chi_sq_min + 1 # using minimum of parabola
  
    # Find the intersection using root_scalar
    intersections = []
    for k in range(len(tc1) - 1):
        if (chi_sq[k] - err_threshold) * (chi_sq[k + 1] - err_threshold) < 0:
            sol = root_scalar(intersection_func, bracket=[tc1[k], tc1[k + 1]])
            if sol.converged:
                intersections.append((sol.root - min_chi_time))
    errors.append(intersections)

    intersections_p = []
    for k in range(len(tc1) - 1):
        if (p_1[k] - err_threshold_p) * (p_1[k + 1] - err_threshold_p) < 0:
            sol = root_scalar(intersection_func, bracket=[tc1[k], tc1[k + 1]])
            if sol.converged:
                intersections_p.append((sol.root - tc_best_fit))
    errors_p.append(intersections_p)

    # plt.plot(tc1[chi_mask], chi_sq[chi_mask],label='chisq')
    # plt.plot(tc1[chi_mask], p_1[chi_mask],label='chisq parabola', color='orange')
    # plt.axvline(x=tc_guess[j], color='r', linestyle='--', label='Bls Guess')
    # plt.axvline(x=min_chi_time, linestyle='--', label='Chisq min')
    # plt.axvline(x=tc1[np.argmin(p_1)], color='orange', linestyle='--', label='Chisq min parabola')

    # # for inter in intersections:
    # #     plt.axvline(x=inter, color='blue', linestyle='--')
    # plt.axhline(y=err_threshold, color='green', linestyle='--', label='Error Threshold')
    # plt.title(f'Transit {j+1}: Planet b')
    # plt.xlabel('tc')
    # plt.ylabel('X^2')
    # plt.legend()
    # plt.show()

In [32]:
#avg the errors   sig^2 = 0.5(sig1^2 + sig2^2)
err_tc_chi = []
for i in range(len(errors)):
    sig = np.sqrt(errors[i][0]**2 + errors[i][1]**2)
    err_tc_chi.append(sig)

err_tc_chi_p = []
for i in range(len(errors_p)):
    sig = np.sqrt(errors_p[i][0]**2 + errors_p[i][1]**2)
    err_tc_chi_p.append(sig)

### Saving Results

In [33]:
# Initialize empty list for storing results
results = []

# Loop through the transits and collect the data dynamically
for j in range(len(tc)):
    transit = transit_num[j]
    transit_time = tc_chi[j]
    avg_error = err_tc_chi[j]
    ttv_value = ttv[j]
    transit_time_parabola = tc_chi_parabola[j]
    avg_error_parabola = err_tc_chi_p[j]
    ttv_value_parabola = ttv_p[j]
    
    # Create a dictionary for the current row
    row = {
        'Planet': 'k2-19b',
        'Transit': transit,
        'Tc(TESS)': transit_time,
        'Tc_err': avg_error,
        'TTV(TESS)': ttv_value,
        'Tc(TESS) Parabola': transit_time_parabola,
        'Tc_err Parabola': avg_error_parabola,
        'TTV(TESS) Parabola': ttv_value_parabola,
        'Time Offset': 'BJD - 2454833',
        'Period': period_b_bls,
        'Tc_offset': tc_b_bls
    }

    # Append the row to the results list
    results.append(row)

# After the loop, create a DataFrame from the results list
tess_transit_data = pd.DataFrame(results)

# Save the DataFrame to a file named 'tess_times.csv'
tess_transit_data.to_csv('tess_transit_data.csv', index=False)

# Print the DataFrame to see the collected data
print(tess_transit_data)

   Planet  Transit     Tc(TESS)    Tc_err  TTV(TESS)  Tc(TESS) Parabola  \
0  k2-19b        0  4697.289029  0.004384   0.008258        4697.288910   
1  k2-19b        2  4713.127344  0.002837   0.005589        4713.127453   
2  k2-19b        3  4721.041830  0.003013  -0.000417        4721.041738   
3  k2-19b        4  4728.959653  0.002993  -0.003086        4728.959826   
4  k2-19b        5  4736.880812  0.004142  -0.002419        4736.880952   
5  k2-19b        6  4744.798635  0.002808  -0.005088        4744.798760   
6  k2-19b       93  5433.878774  0.003929  -0.007758        5433.878892   
7  k2-19b       95  5449.720258  0.002277  -0.007257        5449.720286   

   Tc_err Parabola  TTV(TESS) Parabola    Time Offset    Period  \
0         0.004375            0.008139  BJD - 2454833  7.920492   
1         0.002829            0.005698  BJD - 2454833  7.920492   
2         0.003008           -0.000509  BJD - 2454833  7.920492   
3         0.002976           -0.002913  BJD - 2454833  7